In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!git clone -q https://github.com/coleman-zachery/poker_pytorch.git git_download
!rm -rf libs
!mv git_download/libs .
!rm -rf git_download

from libs.constants import RANKS, SUITS
from libs.helpers import card_strings_to_tensor, card_str_to_tuple, classify_poker_hand, poker_hand_label_to_index, reverse_poker_hand_index
from libs.encoders import CardEncoder, PokerHandClassifier
from libs.generators import generate_straight_flush, generate_four_of_a_kind

In [ ]:
!cat libs/helpers.py

In [ ]:
generate_four_of_a_kind()

In [ ]:
generate_straight_flush()

In [25]:
# ---- training set ----
card_strings = [
    ["A♥", "K♦", "Q♣", "J♠", "__"], # high card
    ["A♥", "K♥", "Q♥", "J♥", "X♥"], # royal flush
    ["9♠", "8♠", "7♠", "6♠", "5♠"], # straight flush
    ["3♦", "3♣", "3♥", "3♠", "9♥"], # four of a kind
    ["4♥", "4♠", "4♣", "9♦", "9♥"], # full house
    ["2♣", "5♣", "9♣", "J♣", "K♣"], # flush
    ["6♥", "5♠", "4♦", "3♣", "2♥"], # straight
    ["7♠", "7♥", "7♣", "2♦", "5♥"], # three of a kind
    ["8♦", "8♣", "4♥", "4♠", "K♥"], # two pair
    ["J♣", "J♠", "3♥", "6♦", "9♣"], # one pair
    ["5♥", "X♠", "7♣", "4♦", "2♥"], # high card
    ["5♥", "5♠", "__", "__", "__"], # one pair with 3 empty cards
    ["9♣", "9♦", "4♥", "4♠", "??"], # two pair and 1 ?? card
    ["__", "__", "__", "__", "__"], # empty hand
]


In [33]:
# create tensors and labels
card_tensors = []
labels = []
for hand in card_strings:
    card_tensor = card_strings_to_tensor(hand)
    card_tensors.append(card_tensor)
    card_tuples = [card_str_to_tuple(cs) for cs in hand]
    label = classify_poker_hand(card_tuples)
    labels.append(poker_hand_label_to_index(label))

# get classification embeddings
card_tensors = torch.stack(card_tensors)
labels = torch.tensor(labels, dtype=torch.long)
# print("Card Tensors:\n", card_tensors)
# print("Labels:\n", labels)
encoder = CardEncoder()
classifier = PokerHandClassifier() # ---- choose classifier ----
encoded_cards = encoder(card_tensors.view(-1, 2))
encoded_hands = encoded_cards.view(card_tensors.size(0), 5, -1)
outputs = classifier(encoded_hands)
print("Classifier Outputs:\n", outputs)

[12, 13, 14, 15]
[11, 12, 13, 14, 15]
[6, 7, 8, 9, 10]
[4, 4, 4, 4, 10]
[5, 5, 5, 10, 10]
[3, 6, 10, 12, 14]
[3, 4, 5, 6, 7]
[3, 6, 8, 8, 8]
[5, 5, 9, 9, 14]
[4, 7, 10, 12, 12]
[3, 5, 6, 8, 11]
[6, 6]
[5, 5, 10, 10]
[]
Classifier Outputs:
 tensor([[ 2.9165e-01,  6.0632e-02, -6.9606e-04,  2.6129e-01, -1.1497e-01,
         -1.9963e-01,  1.2825e-01,  5.1799e-02, -2.4218e-01,  1.7186e-01,
          1.8369e-01],
        [ 1.8140e-01,  9.0543e-02,  1.1982e-01,  1.3976e-01, -1.4759e-01,
         -5.0064e-02,  1.7819e-01,  4.6563e-02, -2.7563e-01,  2.2113e-01,
          1.5047e-01],
        [ 3.4620e-01, -1.1518e-02, -1.5888e-01,  2.2823e-01,  2.0980e-02,
         -3.7492e-01,  6.8279e-02, -1.2948e-01, -3.5245e-01,  3.0044e-01,
          3.9237e-01],
        [ 2.5205e-02,  1.5941e-01,  1.5129e-02,  1.1488e-02, -4.7718e-02,
         -1.5269e-01,  1.7334e-01,  2.3276e-01, -2.4976e-01,  2.4725e-01,
          2.3097e-01],
        [ 2.8684e-01,  8.6315e-02, -1.6107e-01,  2.8941e-01, -1.7513e-01,
  

In [34]:
# ---- train loop ----
def train_loop(model, data_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_cards, batch_labels in data_loader:
            optimizer.zero_grad()
            encoded_cards = encoder(batch_cards.view(-1, 2))
            encoded_hands = encoded_cards.view(batch_cards.size(0), 5, -1)
            outputs = model(encoded_hands)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Example of setting up a data loader and training
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(card_tensors, labels)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(classifier.parameters(), lr=0.001)
train_loop(classifier, data_loader, criterion, optimizer, num_epochs=100)


Epoch [1/100], Loss: 2.4188
Epoch [2/100], Loss: 2.3542
Epoch [3/100], Loss: 2.2790
Epoch [4/100], Loss: 2.2570
Epoch [5/100], Loss: 2.2211
Epoch [6/100], Loss: 2.1405
Epoch [7/100], Loss: 2.1413
Epoch [8/100], Loss: 2.0563
Epoch [9/100], Loss: 2.0280
Epoch [10/100], Loss: 1.9877
Epoch [11/100], Loss: 1.9519
Epoch [12/100], Loss: 1.8625
Epoch [13/100], Loss: 1.8332
Epoch [14/100], Loss: 1.7989
Epoch [15/100], Loss: 1.7414
Epoch [16/100], Loss: 1.6419
Epoch [17/100], Loss: 1.5897
Epoch [18/100], Loss: 1.5596
Epoch [19/100], Loss: 1.4776
Epoch [20/100], Loss: 1.4830
Epoch [21/100], Loss: 1.3908
Epoch [22/100], Loss: 1.2725
Epoch [23/100], Loss: 1.2488
Epoch [24/100], Loss: 1.2319
Epoch [25/100], Loss: 1.1796
Epoch [26/100], Loss: 1.0950
Epoch [27/100], Loss: 1.0755
Epoch [28/100], Loss: 1.0110
Epoch [29/100], Loss: 0.9798
Epoch [30/100], Loss: 0.8982
Epoch [31/100], Loss: 0.8521
Epoch [32/100], Loss: 0.8605
Epoch [33/100], Loss: 0.7619
Epoch [34/100], Loss: 0.7620
Epoch [35/100], Loss: 0

In [35]:
# ---- function that outputs the softmax probability for each class >= 1.00%, formatted to .00% and sorted largest to smallest % ----
def get_class_probabilities(model, hand_tensor):
    model.eval()
    with torch.no_grad():
        encoded_cards = encoder(hand_tensor.view(-1, 2))
        encoded_hand = encoded_cards.view(1, 5, -1)
        outputs = model(encoded_hand)
        probabilities = F.softmax(outputs, dim=1).squeeze(0)
        significant_probs = {reverse_poker_hand_index(i): prob.item() for i, prob in enumerate(probabilities)}
        sorted_probs = dict(sorted(significant_probs.items(), key=lambda item: item[1], reverse=True))
        formatted_probs = {k: f"{v*100:.2f}%" for k, v in sorted_probs.items()}
        return formatted_probs

In [37]:
# ---- print tensor hands, expected output, and classifier outputs ----
for i in range(card_tensors.size(0)):
    hand_tensor = card_tensors[i].unsqueeze(0)
    expected_label = labels[i].item()
    hand_type = reverse_poker_hand_index(expected_label)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    #predicted_label = torch.argmax(outputs, dim=1).item()
    hand = card_strings[i]
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")

# ---- test to see if the model can correctly predict each royal flush hand ----
royal_flush_hands = [
    ["A♣", "K♣", "Q♣", "J♣", "X♣"],
    ["A♦", "K♦", "Q♦", "J♦", "X♦"],
    ["A♥", "K♥", "Q♥", "J♥", "X♥"],
    ["A♠", "K♠", "Q♠", "J♠", "X♠"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    hand_type = "royal flush"
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")



hand: ['A♥', 'K♦', 'Q♣', 'J♠', '__']
correct label confidence: high card -- 92.81%

hand: ['A♥', 'K♥', 'Q♥', 'J♥', 'X♥']
correct label confidence: straight flush -- 93.24%

hand: ['9♠', '8♠', '7♠', '6♠', '5♠']
correct label confidence: straight flush -- 96.72%

hand: ['3♦', '3♣', '3♥', '3♠', '9♥']
correct label confidence: four of a kind -- 95.91%

hand: ['4♥', '4♠', '4♣', '9♦', '9♥']
correct label confidence: full house -- 94.31%

hand: ['2♣', '5♣', '9♣', 'J♣', 'K♣']
correct label confidence: flush -- 97.27%

hand: ['6♥', '5♠', '4♦', '3♣', '2♥']
correct label confidence: straight -- 95.61%

hand: ['7♠', '7♥', '7♣', '2♦', '5♥']
correct label confidence: three of a kind -- 95.30%

hand: ['8♦', '8♣', '4♥', '4♠', 'K♥']
correct label confidence: two pair -- 97.89%

hand: ['J♣', 'J♠', '3♥', '6♦', '9♣']
correct label confidence: one pair -- 97.43%

hand: ['5♥', 'X♠', '7♣', '4♦', '2♥']
correct label confidence: high card -- 97.82%

hand: ['5♥', '5♠', '__', '__', '__']
correct label confidenc

In [31]:
royal_flush_hands = [
    ["__", "A♣", "K♦", "Q♥", "J?"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"class probabilities: {probabilities}")


hand: ['__', 'A♣', 'K♦', 'Q♥', 'J?']
class probabilities: {'nothing': '42.89%', 'one pair': '18.03%', 'straight': '16.22%', 'two pair': '9.52%', 'full house': '4.20%', 'straight flush': '3.82%', 'flush': '2.03%', 'royal flush': '1.00%', 'three of a kind': '0.92%', 'four of a kind': '0.72%', 'high card': '0.65%'}
